<a href="https://colab.research.google.com/github/hukim1112/one-day-LLM/blob/main/LLM_serving_with_vllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

vLLM은 LLM(대규모 언어 모델) 추론 및 서빙을 위한 빠르고 사용하기 쉬운 라이브러리입니다. 이 라이브러리는 다음과 같은 특징을 가지고 있습니다:

- 높은 serving 처리량: vLLM은 높은 서빙 처리량을 제공하여, 대규모 언어 모델을 효과적으로 활용할 수 있습니다.
- PagedAttention: vLLM은 PagedAttention 기술을 사용하여 언어 모델의 'attention' 메커니즘에서 사용하는 키(key)와 값(value)의 메모리를 효율적으로 관리합니다. 이는 메모리 사용을 최적화하고, 더 큰 모델을 더 적은 자원으로 처리할 수 있게 해 줍니다.
- continuous batching: 이 라이브러리는 들어오는 요청을 연속적으로 배치 처리하여, 처리 효율을 높이고 응답 시간을 단축합니다.
- 최적화된 CUDA 커널: vLLM은 GPU에서의 실행을 최적화하기 위해 특별히 설계된 CUDA 커널을 사용합니다. 이를 통해 NVIDIA GPU를 활용하는 시스템에서 더 빠르고 효율적인 성능을 발휘할 수 있습니다.

필요한 라이브러리를 설치합니다.

In [ ]:
!pip3 install transformers
!pip3 install accelerate
!pip3 install vllm
!pip install openai

# Offline Batched Inference

데이터셋에서 오프라인 일괄 추론을 사용하는 예를 먼저 보여드립니다. 즉, 입력 프롬프트 목록에 대해 텍스트를 생성하기 위해 vLLM을 사용합니다.

vLLM에서 LLM과 SamplingParams를 임포트합니다. SamplingParams 클래스는 샘플링 과정의 매개변수를 지정합니다. 생성을 위한 입력 프롬프트 목록과 샘플링 매개변수를 정의합니다. 샘플링 온도는 0.8로 설정되고, nucleus sampling 확률은 0.95로 설정됩니다. 샘플링 매개변수에 대한 자세한 정보는 [클래스 정의](https://github.com/vllm-project/vllm/blob/main/vllm/sampling_params.py)를 참조하세요

In [ ]:
from vllm import LLM, SamplingParams

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

vLLM의 엔진을 LLM 클래스와 OPT-125M 모델을 사용하여 오프라인 추론을 위해 초기화합니다. 지원되는 모델 목록은 [지원 모델](https://docs.vllm.ai/en/latest/models/supported_models.html#supported-models)에서 확인할 수 있습니다.

In [ ]:
llm = LLM(model="facebook/opt-125m")

llm.generate를 호출하여 출력을 생성합니다. 입력 프롬프트를 vLLM 엔진의 대기 큐에 추가하고, 고성능 처리량으로 출력을 생성하기 위해 vLLM 엔진을 실행합니다. 출력은 모든 출력 토큰을 포함하는 RequestOutput 객체의 목록으로 반환됩니다.

In [ ]:
outputs = llm.generate(prompts, sampling_params)

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

In [ ]:
import time

prompt = "Generate a python code that accepts a list of numbers and returns the sum."

start = time.time()
response = llm.generate(prompt, sampling_params)
end = time.time()

latency = end-start
print(f"Latency: {latency}seconds")

output_tokens = len(response[0].outputs[0].token_ids)

through_put = output_tokens / latency

print(f"Throughput: {through_put}tokens/second")

generated_text = response[0].outputs[0].text
print(generated_text)

다음과 같이 gpu 메모리를 반환합니다.

In [ ]:
from vllm.distributed.parallel_state import destroy_model_parallel
import gc, torch
...
destroy_model_parallel()
del llm.llm_engine.model_executor.driver_worker
del llm # Isn't necessary for releasing memory, but why not
gc.collect()
torch.cuda.empty_cache()

# OpenAI 호환 서버

vLLM은 OpenAI API 프로토콜을 구현하는 서버로 배포될 수 있습니다. 이를 통해 OpenAI API를 사용하는 애플리케이션을 위한 드롭인 대체제로 vLLM을 사용할 수 있습니다. 기본적으로 서버는 http://localhost:8000에서 시작됩니다. 주소는 --host 및 --port 인수로 지정할 수 있습니다. 서버는 현재 한 번에 하나의 모델을 호스트합니다(아래 명령에서는 TinyLlama/TinyLlama-1.1B-Chat-v1.0)이며 모델 목록, 채팅 완성 생성 및 완성 생성 엔드포인트를 구현합니다. 더 많은 엔드포인트 지원을 활발히 추가하고 있습니다.

```
python -m vllm.entrypoints.openai.api_server \
--model TinyLlama/TinyLlama-1.1B-Chat-v1.0 --dtype=half

```

In [ ]:
!curl http://localhost:8000/v1/models

기본적으로 서버는 토크나이저에 저장된 미리 정의된 채팅 템플릿을 사용합니다. 이 템플릿은 --chat-template 인수를 사용하여 덮어쓸 수 있습니다:

```
!wget https://github.com/vllm-project/vllm/blob/main/examples/template_chatml.jinja
!python -m vllm.entrypoints.openai.api_server \
--model NousResearch/Meta-Llama-3-8B-Instruct \
--chat-template ./examples/template_chatml.jinja
```

# vLLM으로 OpenAI completion API 사용하기

다음 입력 프롬프트를 질의합니다.

In [ ]:
!curl http://localhost:8000/v1/completions \
-H "Content-Type: application/json" \
-d '{ "model": "TinyLlama/TinyLlama-1.1B-Chat-v1.0", "prompt": "San Francisco is a", "max_tokens": 7, "temperature": 0 }'

vllm 서버는 OpenAI API와 호환되므로 OpenAI API를 사용하는 모든 애플리케이션에서 대체제로 사용할 수 있습니다. 예를 들어, 이 서버를 쿼리하는 다른 방법은 openai 파이썬 패키지를 통한 것입니다:

In [ ]:
from openai import OpenAI

# OpenAI의 API 키와 API 기본을 vLLM의 API 서버로 사용하도록 수정합니다.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                      prompt="San Francisco is a")
print("Completion result:", completion)

In [ ]:
completion.choices[0].text

# vLLM으로 OpenAI chat API 사용하기

vLLM 서버는 OpenAI Chat API를 지원하도록 설계되었으며, 이를 통해 모델과 대화를 나눌 수 있습니다. 채팅 인터페이스는 모델과의 소통을 더 상호작용적인 방식으로 가능하게 해주며, 채팅 기록에 저장될 수 있는 왕복 교환을 허용합니다. 이는 맥락이 필요하거나 보다 자세한 설명이 요구되는 작업에 유용합니다.

OpenAI Chat API를 사용하여 모델을 쿼리하는 방법:

채팅과 같은 인터페이스에서 모델과 소통할 수 있도록 채팅 완성 엔드포인트를 사용할 수 있습니다:

In [ ]:
!curl http://localhost:8000/v1/chat/completions \
-H "Content-Type: application/json" \
-d '{"model": "TinyLlama/TinyLlama-1.1B-Chat-v1.0", "messages": [ {"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Who won the world series in 2020?"}]}'

In [ ]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ]
)
print("Chat response:", chat_response)

In [ ]:
print("Chat response:", chat_response.choices[0].message.content)

다음은 지속적인 대화를 유지하기 위한 샘플 코드:

In [ ]:
from openai import OpenAI

# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# 초기 메시지 설정
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is your name?"},
]

# 모델과 대화 시작
chat_response = client.chat.completions.create(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    messages=messages
)

print("Chat response:", chat_response)

# 새로운 메시지 추가
assistant_message = chat_response.choices[0].message.content
messages.append({"role": "assistant", "content": assistant_message})
messages.append({"role": "user", "content": "Can you tell me a joke?"})

# 다시 모델과 대화
chat_response = client.chat.completions.create(
    model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    messages=messages
)

print("Chat response:", chat_response)

# 대화 히스토리 업데이트
assistant_message = chat_response.choices[0].message.content
messages.append({"role": "assistant", "content": assistant_message})


In [ ]:
messages

# Reference
- https://docs.vllm.ai/en/latest/getting_started/quickstart.html
- https://towardsdatascience.com/deploying-large-language-models-vllm-and-quantizationstep-by-step-guide-on-how-to-accelerate-becfe17396a2
- https://blog.vllm.ai/2023/06/20/vllm.html?source=post_page-----becfe17396a2--------------------------------